In [ ]:
"""
testing.ipynb

File for performing testing to implement lottery ticket experiments.

Authors: Jordan Bourdeau, Casey Forey
Date Created: 3/8/24
"""

%load_ext tensorboard
import functools
from importlib import reload
import numpy as np
import os
import random
import tensorflow as tf
import tensorflow_model_optimization as tfmot
from tensorflow import keras
from keras.callbacks import Callback
from keras import backend as K
from keras import Sequential
from keras.layers import Dense, Input
from keras.losses import CategoricalCrossentropy

from tensorflow_model_optimization.sparsity import keras as sparsity
from tensorflow_model_optimization.sparsity.keras import ConstantSparsity, PolynomialDecay, prune_low_magnitude

from src.harness.constants import Constants as C
from src.harness.dataset import download_data, load_and_process_mnist
from src.harness.experiment import ExperimentData
from src.harness.model import create_lenet_300_100, create_masked_nn, create_pruned_lenet, save_model, load_model
from src.harness.pruning import create_pruning_callbacks, create_pruning_parameters
from src.harness.training import test_step, train, train_one_step, training_loop, TrainingRound
from src.harness.utils import count_params, set_seed

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

## Parameters

In [ ]:
X_train, X_test, Y_train, Y_test = load_and_process_mnist()

num_epochs: int = 10
batch_size: int = 60
input_shape: tuple = X_train[0].shape
num_classes: int = 10

num_train_samples: int = X_train.shape[0]

end_step: int = np.ceil(1.0 * num_train_samples / batch_size).astype(np.int32) * num_epochs

pruning_parameters: dict = create_pruning_parameters(0.01, 0, end_step, 100)

## Building

In [ ]:
# Create a model with the same architecture using all Keras components to check its accuracy with the same parameters
set_seed(0)

model: keras.Model = create_lenet_300_100(input_shape, num_classes)
model.summary()
model.trainable_variables

In [ ]:
mask_model: keras.Model = create_masked_nn(create_pruned_lenet, input_shape, num_classes, pruning_parameters)
mask_model.summary()
mask_model.trainable_variables

## Pruning

## Training

In [ ]:
# Test single step of training

# Define the optimizer outside of the function
optimizer = C.OPTIMIZER()

# Create model and masked model
model: keras.Model = create_lenet_300_100(input_shape, num_classes)
mask_model: keras.Model = create_masked_nn(create_pruned_lenet, input_shape, num_classes, pruning_parameters)

loss, accuracy = train_one_step(model, mask_model, X_train, Y_train, optimizer)
print(f'Loss: {loss:.6f}, Accuracy: {accuracy:.6g}')

In [ ]:
# Try getting test accuracies with `test_step`
test_loss: tf.keras.metrics.Metric = C.LOSS_FUNCTION()
test_accuracy: tf.keras.metrics.Metric = tf.keras.metrics.CategoricalAccuracy()

loss, accuracy = test_step(model, X_test, Y_test, test_loss, test_accuracy)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

In [ ]:
# Testing `training_loop` function

# Define the optimizer outside of the function
optimizer = C.OPTIMIZER()

epochs: int = 2

# Create model and masked model
model: keras.Model = create_lenet_300_100(input_shape, num_classes)
mask_model: keras.Model = create_masked_nn(create_pruned_lenet, input_shape, num_classes, pruning_parameters)

trained_model, training_round = training_loop(0, model, mask_model, load_and_process_mnist, epochs, batch_size, optimizer=optimizer)
print(f'Took {np.sum(training_round.test_accuracies != 0)} / {epochs} epochs')
print(f'Ended with a best training accuracy of {np.max(training_round.train_accuracies) * 100:.2f}% and test accuracy of {np.max(training_round.test_accuracies) * 100:.2f}%')

In [ ]:
# Testing `train` function

# Create model and masked model
model: keras.Model = create_lenet_300_100(input_shape, num_classes)
mask_model: keras.Model = create_masked_nn(create_pruned_lenet, input_shape, num_classes, pruning_parameters)

trained_model, pruned_mask_model, training_round = train(0, 0, model, mask_model, load_and_process_mnist)

test_accuracy: tf.keras.metrics.Metric = tf.keras.metrics.CategoricalAccuracy()

print(f'Took {np.sum(training_round.test_accuracies != 0)} / {C.TRAINING_EPOCHS} epochs')
print(f'Ended with a best training accuracy of {np.max(training_round.train_accuracies) * 100:.2f}% and test accuracy of training accuracy of {np.max(training_round.test_accuracies) * 100:.2f}%')

loss, accuracy = test_step(trained_model, X_test, Y_test, test_loss, test_accuracy)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

In [ ]:
# Test loading the model back
loaded_model: keras.Model = load_model(0, 0)

loss, accuracy = test_step(loaded_model, X_test, Y_test, test_loss, test_accuracy)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

In [ ]:
print(f'Nonzero parameters after training:')
print(count_params(model)[1])